# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [1]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [8]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


biology_template = """你是一位资深的生物学专家。你擅长回答生物学问题。
你了解生物学众多领域的大量信息，并且你阅读和理解过大量科学文献，
在回答问题时，你会结合科学文献进行解释.

这是一个问题：
{input}"""


computer_template = """你是一位计算机专家。你擅长回答计算机相关的问题。
你已经拥有了很多年的开发经验，并且你懂得查阅技术论坛
在回答问题时，你会根据经验以及整合查阅的结果进行分析推理

这是一个问题：
{input}"""


chinese_language_template = """你是一位汉语言文学老师。你擅长回答汉语言文学问题。
你精通汉语言文学领域，阅读过大量的中文经典作品
你在评价作品的时候，会结合历史背景进行评价

这是一个问题：
{input}"""

In [9]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
	{
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
	{
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
	{
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_language_template,
    }
]

In [10]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [11]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [12]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [13]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [14]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [15]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言文学: 适用于回答汉语言文学问题']


In [16]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题


In [17]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [18]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

<

In [19]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [20]:
print(
    chain.run(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)

D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数'}
> Finished chain.
是多少？

首先，我们需要知道质数是什么。质数是只能被1和自身整除的正整数。
接下来，我们需要列出大于40的所有正整数：41、42、43、44、45、46、47、48、49、50…
然后，我们一个一个地检查这些数是否为质数。
我们发现41是大于40的第一个质数，因为它只能被1和41整除，没有其他因数。
因此，大于40的第一个质数是41。


In [21]:
router_chain.verbose = True

In [17]:
print(chain.run("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.


黑洞是宇宙中一种非常密集的天体，其质量非常巨大，但体积非常小。它的引力非常强大，甚至连光也无法逃离它的吸引力。这是因为黑洞的质量曲折了时空，使光线弯曲，从而让它看起来像一个黑暗的球体。黑洞的形成是因为某些恒星在死亡后，其质量会塌缩到极限，形成一个无法逃脱的引力陷阱。我们目前对黑洞的了解仍然不够完整，但它们在宇宙中扮演着重要的角色，影响着星系的演化。希望我的回答能够解决您的疑问。


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [22]:
print(chain.run("介绍一下http协议"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '什么是http协议'}
> Finished chain.


HTTP（Hypertext Transfer Protocol）协议是一种用于传输超文本的应用层协议，它是Web上数据传输的基础。它的主要作用是规定客户端和服务器之间进行通信的标准，使得网络上的客户端可以与服务器进行交互，获取网页内容或其他资源。HTTP协议使用TCP作为它的传输层协议，通过HTTP请求和响应的方式来传输数据。在Web浏览器中输入网址、点击链接或提交表单时，就会发起HTTP请求，服务器接收到请求后会响应相应的数据。HTTP协议也被广泛应用于其他应用程序之间的通信，如API接口的调用等。


In [23]:
print(chain.invoke("细胞核是什么?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '细胞核是什么?'}
> Finished chain.
{'input': '细胞核是什么?', 'text': '\n\n细胞核是细胞的一个重要组成部分，它是一个包含细胞遗传物质DNA的膜包围的结构。细胞核负责细胞的遗传信息的储存和传递，控制细胞的生长、分裂和特殊功能的发挥。除了DNA，细胞核还含有RNA、蛋白质和其他重要的生物分子。细胞核在细胞内也起到了保护和维持细胞内平衡的作用。它还具有调节细胞代谢活动、细胞信号传导和细胞命运决定的重要作用。总的来说，细胞核是细胞的控制中心，能够维持细胞的正常功能并确保细胞的生存。'}
